In [2]:
# Install Java
!apt-get update
!apt-get install -y openjdk-11-jdk-headless
!java -version

# Download Stanford CoreNLP 4.5.5
!wget https://nlp.stanford.edu/software/stanford-corenlp-4.5.5.zip
!unzip stanford-corenlp-4.5.5.zip

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:3 https://packages.cloud.google.com/apt gcsfuse-focal InRelease [1227 B]
Get:4 https://packages.cloud.google.com/apt cloud-sdk InRelease [1618 B]       
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]        
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]
Get:7 https://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [28.6 kB]
Get:8 https://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [3339 kB]
Get:9 https://packages.cloud.google.com/apt cloud-sdk/main all Packages [1552 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [4069 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [4242 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [4090 kB]
Get:13 http://security.ubuntu.com/ubun

In [3]:
!wget http://nlp.stanford.edu/software/stanford-corenlp-4.5.5-models-english.jar

--2024-10-22 04:59:04--  http://nlp.stanford.edu/software/stanford-corenlp-4.5.5-models-english.jar
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/software/stanford-corenlp-4.5.5-models-english.jar [following]
--2024-10-22 04:59:05--  https://nlp.stanford.edu/software/stanford-corenlp-4.5.5-models-english.jar
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://downloads.cs.stanford.edu/nlp/software/stanford-corenlp-4.5.5-models-english.jar [following]
--2024-10-22 04:59:05--  https://downloads.cs.stanford.edu/nlp/software/stanford-corenlp-4.5.5-models-english.jar
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|

In [4]:
!export CLASSPATH=$CLASSPATH:/kaggle/working/stanford-corenlp-4.5.5/*:/kaggle/working/stanford-corenlp-4.5.5-models-english.jar

In [5]:
import subprocess

try:
    result = subprocess.run(["jar", "tf", "/kaggle/working/stanford-corenlp-4.5.5/stanford-corenlp-4.5.5.jar"], 
                            capture_output=True, text=True, check=True)
    merge_nodes_entries = [line for line in result.stdout.split('\n') if 'MergeNodes' in line]
    
    if merge_nodes_entries:
        print("MergeNodes entries found:")
        for entry in merge_nodes_entries:
            print(entry)
    else:
        print("No MergeNodes entries found in the JAR file.")
except subprocess.CalledProcessError as e:
    print(f"An error occurred while executing the jar command: {e}")
except FileNotFoundError:
    print("The jar command was not found. Make sure Java is installed and in your PATH.")

MergeNodes entries found:
edu/stanford/nlp/semgraph/semgrex/ssurgeon/MergeNodes.class


In [12]:
import subprocess
import os

java_code = '''

import edu.stanford.nlp.pipeline.*;
import edu.stanford.nlp.semgraph.*;
import edu.stanford.nlp.semgraph.semgrex.*;
import edu.stanford.nlp.ling.*;
import edu.stanford.nlp.util.*;
import edu.stanford.nlp.semgraph.semgrex.ssurgeon.*;
import java.util.*;

public class SemgrexLimitationsTest {
    public static void main(String[] args) {
        Properties props = new Properties();
        props.setProperty("annotators", "tokenize,ssplit,pos,lemma,depparse");
        StanfordCoreNLP pipeline = new StanfordCoreNLP(props);

        testMultiTokenExpressions(pipeline);
        testChainPatterns(pipeline);
    }
    
    private static void testMultiTokenExpressions(StanfordCoreNLP pipeline) {
        System.out.println("\\n=== Testing Multi-token Expression Support ===");
        String text = "He looked the new information up carefully yesterday";
        CoreDocument document = new CoreDocument(text);
        pipeline.annotate(document);
        SemanticGraph graph = document.sentences().get(0).dependencyParse();
        
        System.out.println("Graph:\\n" + graph.toString(SemanticGraph.OutputFormat.LIST));
        
        // Test for phrasal verb components
        testPattern(graph, "Match main verb", "{word:looked}");
        testPattern(graph, "Match particle", "{word:up}");
        
        // Test for the intervening object
        testPattern(graph, "Match object with modifiers", 
                   "{word:information} <det {} <amod {}");
        
        // Test relation between verb and particle
        testPattern(graph, "Relation to particle", 
                   "{} >case {word:up}");
    }
    
    private static void testChainPatterns(StanfordCoreNLP pipeline) {
        System.out.println("\\n=== Testing Chain Pattern Support ===");
        String text = "John's friend's brother's sister's house";
        CoreDocument document = new CoreDocument(text);
        pipeline.annotate(document);
        SemanticGraph graph = document.sentences().get(0).dependencyParse();
        
        System.out.println("Graph:\\n" + graph.toString(SemanticGraph.OutputFormat.LIST));
        
        // Test single possessive
        testPattern(graph, "Single possessive", 
                   "{} >nmod:poss {}");
        
        // Test two-level chain without parens
        testPattern(graph, "Two-level chain", 
                   "{} >nmod:poss {} >nmod:poss {}");
        
        // Test for finding the full chain's end points
        testPattern(graph, "Chain endpoints", 
                   "{} >nmod:poss+ {}");
    }
    
    private static void testPattern(SemanticGraph graph, String description, String pattern) {
        try {
            System.out.println("\\nTesting: " + description);
            System.out.println("Pattern: " + pattern);
            SemgrexPattern semgrexPattern = SemgrexPattern.compile(pattern);
            SemgrexMatcher matcher = semgrexPattern.matcher(graph);
            boolean found = false;
            while (matcher.find()) {
                found = true;
                System.out.println("Match: " + matcher.getMatch());
            }
            if (!found) {
                System.out.println("No matches found");
            }
        } catch (Exception e) {
            System.out.println("Error: " + e.getMessage());
        }
    }
}
     
'''

with open('SemgrexLimitationsTest.java', 'w') as f:
    f.write(java_code)

# Compile the Java code
compile_command = ["javac", "-encoding", "UTF-8", "-cp", ".:/kaggle/working/stanford-corenlp-4.5.5/*", "SemgrexLimitationsTest.java"]
compile_result = subprocess.run(compile_command, capture_output=True, text=True)

if compile_result.returncode == 0:
    print("Compilation successful")
    
    # Run the Java program
    run_command = ["java", "-cp", ".:/kaggle/working/stanford-corenlp-4.5.5/*", "SemgrexLimitationsTest"]
    run_result = subprocess.run(run_command, capture_output=True, text=True)
    
    print("Program output:")
    print(run_result.stdout)
    
    if run_result.stderr:
        print("Errors or warnings:")
        print(run_result.stderr)
else:
    print("Compilation failed:")
    print(compile_result.stderr)

Compilation successful
Program output:

=== Testing Multi-token Expression Support ===
Graph:
root(ROOT-0, looked-2)
nsubj(looked-2, He-1)
det(information-5, the-3)
amod(information-5, new-4)
obj(looked-2, information-5)
case(carefully-7, up-6)
advmod(looked-2, carefully-7)
obl:tmod(looked-2, yesterday-8)


Testing: Match main verb
Pattern: {word:looked}
Match: looked/VBD

Testing: Match particle
Pattern: {word:up}
Match: up/IN

Testing: Match object with modifiers
Pattern: {word:information} <det {} <amod {}
No matches found

Testing: Relation to particle
Pattern: {} >case {word:up}
Match: carefully/RB

=== Testing Chain Pattern Support ===
Graph:
root(ROOT-0, house-9)
nmod:poss(friend-3, John-1)
case(John-1, 's-2)
nmod:poss(brother-5, friend-3)
case(friend-3, 's-4)
nmod:poss(sister-7, brother-5)
case(brother-5, 's-6)
nmod:poss(house-9, sister-7)
case(sister-7, 's-8)


Testing: Single possessive
Pattern: {} >nmod:poss {}
Error: Error parsing semgrex pattern {} >nmod:poss {}

Testing: 